# <center>Lecture 1: introduction to optimal transport: the discrete case</center>
### <center>Alfred Galichon (NYU)</center>
## <center>14th European Summer School in Financial Mathematics</center>
<center>© 2021 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274, and contributions by Jules Baudet, Pauline Corblet, Gregory Dannay, Julie Lenoir, and James Nesbit are acknowledged.</center>

A large part of this material is taken from the math+econ+code lecture series:<br>
https://www.math-econ-code.org/

# Part 1: discrete optimal transport and linear programming

### Learning Objectives

* Optimal assignment problem

* Pairwise stability, Walrasian equilibrium

* Computation

### References

* Galichon, *Optimal Transport Methods in Economics*, Ch. 3

* Roth, Sotomayor(1990). *Two-Sided Matching*. Cambridge.

* Koopmans and Beckmann (1957). "Assignment problems and the location of economic activities." *Econometrica*.

* Shapley and Shubik (1972). The assignment game I: The core." *IJGT*.

* Becker (1993). *A Treatise of the Family*. Harvard.

* Gretsky, Ostroy, and Zame (1992). "The nonatomic assignment model." *Economic Theory*.

* Burkard, Dell'Amico, and Martello (2012). *Assignment Problems*. SIAM.

* Dupuy and Galichon (2014). "Personality traits and the marriage market." *JPE*.

## Motivation

### Optimal Transport

Consider the problem of assigning a possibly infinite number of workers
and firms.

* Each worker should work for one firm, and each firm should hire one worker.

* Workers and firms have heterogenous characteristics; let $x\in \mathcal{X}$ and $y\in\mathcal{Y}$ be the characteristics of workers and firms respectively.

* Workers and firms are in equal mass, which is normalized to one. The distribution of worker's types is $P$, and the distribution of the firm's types is $Q$, where $P$ and $Q$ are probability measures on $\mathcal{X}$ and $\mathcal{Y}$.

It is assumed that if a worker $x$ matches with a firm $y$, the total output generated is $\Phi_{xy}$. The questions are then:

* **Optimality:** what is the optimal assignment in the sense that it maximizes the overal output generated?

* **Equilibrium:** what are the equilibrium assignment and the equilibrium wages?

* **Efficiency:** do these two notions coincide?

The same tools have been used by Gary Becker to study the heterosexual marriage market, where $x$ is the man's characteristics, and $y$ is the woman's characteristics, and "wages" are replaced by "transfers".


## Data

In this lecture, we shall take a look at marriage data (while a worker-firm example will be seen in next block). Dupuy and Galichon (JPE, 2014) study a marriage dataset where, in addition to usual socio-demographic variables (such as education and age), measures of personality traits are reported.

* The literature on quantitative psychology argues that one can capture relatively well an individual's personality along five dimensions, the "big 5" - consciousness, extraversion, agreableness, emotional stability, autonomy - assessed though a standardized questionaire.

* In addition to this, we observed a (self-assessed) measure of health, risk-aversion, education, height and body mass index = weight in kg/ (height in m)$^2$. In total, the available characteristics $x_{i}$ of man $i$ and $y_{j}$ of woman $j$ are both $10$-dimensional vectors.

* It is assumed that the surplus of interaction is given by $\Phi\left(x_{i},y_{j}\right)  =x_{i}^{\intercal}Ay_{j}$, where $A$ is a *given* $10 \times 10$ matrix. (later in this course, we'll see how to estimate $A$ based on matched marital data).

Today, we solve a central planner's problem (a stylized version of the problem OKCupids would solve): given a population of men and a population of women, how do we mutually assign these in order to:

1. maximize matching surplus 

2. attain a (hopefully) stable assignment.

We load the libraries that we shall need. If you have not Gurobi already installed on your machine, you may load it (with a demo license) using `pip install` as described in the commented line below.

In [1]:
# !python -m pip install -i https://pypi.gurobi.com gurobipy ## only if Gurobi not here

import pandas as pd
import numpy as np
import scipy.sparse as spr
import time
import gurobipy as grb

We load the data. These are the men's and the women's characteristics, and their mutual affinities.

In [2]:
thepath = 'https://raw.githubusercontent.com/math-econ-code/mec_optim_2021-01/master/data_mec_optim/marriage_personality-traits/'

data_X = pd.read_csv(thepath + "Xvals.csv")
data_Y = pd.read_csv(thepath + "Yvals.csv")
affdf = pd.read_csv(thepath + "affinitymatrix.csv")

data_X.head()

,educm,heightm,BMIm,healthm,consm,extram,agreem,emom,autom,riskym
0,2,186,28.905075,3,-0.752877,-0.360787,-0.711276,-0.291031,0.840217,0.479437
1,2,176,27.440599,3,0.345542,-0.805524,-0.251796,-0.305475,-0.064454,0.030303
2,3,187,23.163374,3,-0.759678,0.898007,-0.029462,-0.672859,-0.961691,-0.556598
3,1,184,29.241493,2,-0.455688,-1.053375,-0.041612,0.436133,0.121873,0.992084
4,1,174,23.781214,4,-1.440239,1.163730,0.293750,-0.538922,0.782285,-1.401034


## The Discrete Monge-Kantorovich Theorem

Assume that the type spaces $\mathcal{X}$ and $\mathcal{Y}$ are finite, so $\mathcal{X=}\left\{  1,...,N\right\}  $, and $\mathcal{Y}=\left\{1,...,M\right\}$. 

The mass of workers of type $x$ is $p_{x}$; the mass of jobs of type $y$ is $q_{y}$, with $\sum_{x}p_{x}=\sum_{y}q_{y}=1$.

Let $\pi_{xy}$ be the mass of workers of type $x$ assigned to jobs of type $y$. Every worker is busy and every job is filled, thus

<a name="dicsr-constraints"></a>
\begin{align*}
\pi\in\mathcal{M}\left(p,q\right) :=\left\{ \pi_{xy}\geq 0, \sum_{y\in\mathcal{Y}}\pi_{xy}=p_{x}\text{ and }\sum_{x\in\mathcal{X}}\pi
_{xy}=q_{y} \right\}. 
\end{align*}

(Note that this formulation allows for mixing, i.e. it allows for $\pi_{xy}>0$ and $\pi_{xy^{\prime}}>0$ to hold simultaneously with $y\neq y^{\prime}$.)


Assume the economic output created when assigning worker $x$ to job $y$ is $\Phi_{xy}$. Hence, the optimal assignment is

<a name="OAP"></a>
\begin{align*}
\max_{\pi\geq0}  &  \sum_{xy}\pi_{xy}\Phi_{xy} \\
s.t.~  &  \sum_{y\in\mathcal{Y}}\pi_{xy}=p_{x} \left[u_{x}\right] \\
&  \sum_{x\in\mathcal{X}}\pi_{xy}=q_{y}~\left[v_{y}\right]
\end{align*}

## Duality

---
**Theorem**
1. The value of the [primal problem](#OAP) *coincides with the value of the dual problem*

    <a name="dual-discr"></a>
    \begin{align*}
    \min_{u,v}  &  \sum_{x\in\mathcal{X}}p_{x}u_{x}+\sum_{y\in\mathcal{Y}}
    q_{y}v_{y}\\
    s.t.  &  u_{x}+v_{y}\geq\Phi_{xy}~\left[\pi_{xy}\geq0\right]
    \end{align*}

2. Both the primal and the dual problems have optimal solutions. If $\pi$ *is a solution to the primal problem* and $\left(u,v\right)$ is *a solution to the dual problem, then by complementary slackness*,

    <a name="complSlack"> </a>
    \begin{align*}
    \pi_{xy}>0\text{ implies }u_{x}+v_{y}=\Phi_{xy}.
    \end{align*}


**Proof**. We have: 

1. The value of the [primal problem](#OAP) can be written as $\max_{\pi\geq0}\min_{u,v}S\left(  \pi,u,v\right)$, where

    \begin{align*}
    S\left(  \pi,u,v\right)  :=\sum_{xy}\pi_{xy}\Phi_{xy}+\sum_{x\in\mathcal{X}}u_{x}(p_{x}-\sum_{y\in\mathcal{Y}}\pi_{xy})+\sum_{y\in\mathcal{Y}}v_{y}(q_{y}-\sum_{x\in\mathcal{X}}\pi_{xy})
    \end{align*}

    but by the minmax theorem, this value is equal to $\min_{u,v}\max_{\pi\geq 0}S\left(  \pi,u,v\right)$, which is the value of the [dual problem](#ual-discr}).

2. Follows by noting that, for a primal solution $\pi$ and a dual solution $\left(  u,v\right)  $, then $S\left( \pi,u,v\right)  =\sum_{xy}\pi_{xy} \Phi_{xy}$. $\blacksquare$

**Remarks**. Note that this result is the min-cost flow duality theorem in the bipartite case, as seen in lecture $2$, after setting transportation cost through $xy\in\mathcal{X}\times\mathcal{Y}$ to $c_{xy}=-\Phi_{xy}$, and $n_{t}=-p_{t}1\left\{  t\in\mathcal{X}\right\}  +q_{t}\mathbf{1}\left\{  t\in\mathcal{Y}\right\}$. We see various new interpretations of the result.

The following statements are equivalent:

* $\pi$ is an optimal solution to the primal problem, and $\left(
u,v\right)  $ is an optimal solution to the dual problem, and

* (i) $\pi\in M\left(  p,q\right)  $

* (ii) $u_{x}+v_{y}\geq\Phi_{xy}$

* (iii) $\pi_{xy}>0$ implies $u_{x}+v_{y}\leq\Phi_{xy}$.

We saw the direct implication. But the converse is easy: take $\pi$ and $\left(  u,v\right)  $ satisfying (i)--(iii), Then one has

\begin{align*}
dual\leq\sum_{x}p_{x}u_{x}+\sum_{y}q_{y}v_{y}=\sum_{xy}\pi_{xy}\left(
u_{x}+v_{y}\right)  \leq\sum_{xy}\pi_{xy}\Phi_{xy}\leq primal
\end{align*}

but by the MK duality theorem, both ends coincide. Thus $\pi$ is optimal for the primal and $\left(  u,v\right)  $ for the dual.

### Unassigned Agents

A important variant of the problem exists with $\sum_{x\in\mathcal{X}}p_{x}\neq\sum_{y\in\mathcal{Y}}q_{y}$ and the primal constraints become inequality constraints. The duality then becomes

\begin{align*}
\begin{array}
[c]{rrr}
\max_{\pi\geq0}\sum\pi_{xy}\Phi_{xy} & = & \min_{u,v}\sum_{x\in\mathcal{X}
}p_{x}u_{x}+\sum_{y\in\mathcal{Y}}q_{y}v_{y}\\
s.t.~\sum_{y\in\mathcal{Y}}\pi_{xy}\leq p_{x} &  & u\geq0,~v\geq0 \\
\sum_{x\in\mathcal{X}}\pi_{xy}\leq q_{y} &  & u_{x}+v_{y}\geq\Phi_{xy}
\end{array}
\end{align*}


### Pairwise stability

In a marriage context, an important concept is stability:

An outcome is a vector $\left(  \pi,u,v\right)  $, where $u_{x}$ and $v_{y}$ are $x$'s and $y$'s payoffs, and $\pi$ is a matching that is

<a name="primFeas"></a>
\begin{align*}
\pi\in\mathcal{M}\left(  p,q\right).
\end{align*}

A pair $xy$ is blocking if $x$ and $y$ can find a way of sharing their joint surplus $\Phi_{xy}$ in such a way that $x$ gets more than $u_{x}$ and $y$ gets more than $v_{y}$. Hence there is no blocking pair if and only if for every $x$ and $y$, one has

<a name="noBlocking"></a>
\begin{align*}
u_{x}+v_{y}\geq\Phi_{xy}.
\end{align*}

If $x$ and $y$ are actually matched, their utilities $u_{x}$ and $v_{y}$ need to be feasible, i.e. the above inequality should be saturated. Hence

<a name="cplSlck"></a>
\begin{align*}
\pi_{xy}>0\text{ implies }u_{x}+v_{y}=\Phi_{xy}.
\end{align*}

---
**Definition:**

A matching that satisfies [primal feasbilitity](#primFeas), [no blocking](#noBlocking), and [complementary slackness](#cplSlck) is called a stable matching.

As it turns out, these conditions are precisely the conditions that express complementarity slackness in the Monge-Kantorovich problem. Therefore, outcome$\left(  \pi,u,v\right)  $ is stable if and only if $\pi$ is a solution to the primal problem, and $\left(  u,v\right)  $ is a solution to the dual problem.

### Wage market interpretation 1

Back to the workers / firms interpretation and assume for now that workers are indifferent between any two firms that offer the same salary. We argue that $u\left(  x\right)  $ can be interpreted as the equilibrium wage of worker $x$, while $v\left(  y\right)  $ can be interpreted as the equilibrium profit of firm $y$. Indeed:

---
**Proposition**
If $\left(u,v\right)$ is a solution to the dual of the Kantorovich problem, then

\begin{align*}
u_{x}  &  =\sup_{y\in\mathcal{Y}}\left(  \Phi_{xy}-v_{y}\right)
\label{conjug1}\\
v_{y}  &  =\sup_{x\in\mathcal{X}}\left(  \Phi_{xy}-u_{x}\right)  .
\label{conjug2}
\end{align*}

---

Therefore, $u_{x}$ can be interpreted as equilibrium wage of worker $x$, and $v_{y}$ as equilibrium profit of firm $y$. In this interpretation, all workers get the same wage at equilibrium.


### Wage market interpretation 2

Assume now that if a worker of type $x$ works for a firm of type $y$ for wage $w_{xy}$, then gets $\alpha_{xy}+w_{xy}$, where $\alpha_{xy}$ is the nonmonetary payoff associated with working with a firm of type $y$. The firm's profit is $\gamma_{xy}-w_{xy}$, where $\gamma_{xy}$ is the economic output.

If an employee of type $x$ matches with a firm of type $y$, they generate joint surplus $\Phi_{xy}$, given by%

\begin{align*}
\Phi_{xy}=\underset{\text{employee's payoff}}{\underbrace{\alpha_{xy}+w_{xy}}}+\underset{\text{firm's payoff}}{\underbrace{\gamma_{xy}-w_{xy}}}=\alpha_{xy}+\gamma_{xy}
\end{align*}

which is independent from $w$.

Workers choose firms which maximize their utility, i.e. solve 

<a name="equivWalrasStable1"></a>
\begin{align*}
u_{x}=\max_{y}\left\{  \alpha_{xy}+w_{xy}\right\}
\end{align*}

and $u_{x}=\alpha_{xy}+w_{xy}$ if $x$ and $y$ are matched. Similarly, the indirect payoff vector of firms is

<a name="equivWalrasStable2"></a>
\begin{align*}
v_{y}=\max_{x}\left\{  \gamma_{xy}-w_{xy}\right\}
\end{align*}

and, again, $v_{y}=\gamma_{xy}-w_{xy}$ if $x$ and $y$ are matched.

As a result,

\begin{align*}
u_{x}+v_{y}\geq\alpha_{xy}+\gamma_{xy}=\Phi_{xy}
\end{align*}

and equality holds if $x$ and $y$ are matched. Thus, if $w_{xy}$ is an equilibrium wage, then the triple $\left(  \pi,u,v\right)$ where $\pi$ is the corresponding matching, and $u_{x}$ and $v_{y}$ are defined by [this](#equivWalrasStable1) and [this](#equivWalrasStable2) defines a stable outcome.

Conversely, let $\left(\pi,u,v\right)$ be a stable outcome. Then let $\bar{w}_{xx}$ and \b{w}$_{xy}$ be defined by

\begin{align*}
\bar{w}_{xy}=u_{x}-\alpha_{xy}\text{ and }w^l_{xy}=\gamma_{xy}-v_{y}.
\end{align*}

One has $\bar{w}_{xy}\geq b^l_{xy}$. Any $w_{xy}$ such that $\bar{w}_{xy}\geq w_{xy}\geq b^l_{xy}$ is an equilibrium wage. Indeed, $\pi_{xy}>0$ implies $\bar{w}_{xy} \geq b^l_{xy}$, thus [this](#equivWalrasStable1) and [this](#equivWalrasStable2) hold. Given $u$ and $v$, $w_{xy}$ is uniquely defined on the equilibrium path (ie. when $x$ and $y$ are such that $\pi_{xy}>0$), but there are multiple choices of $w$ outside the equilibrium path.

Note that all workers of the same type get the same indirect utility, but not necessarly the same wage.

## Application: marital matchmaking

We postulate that the form of the surplus function is
\begin{align*}
\Phi_{ij}=x_{i}^{\intercal} Ay_{j}
\end{align*}
where $x_{i}$ and $y_{j}$ are the 10-dimensional characteristics of man $i$ and woman $j$, and the form of $A$, a 10x10 matrix, is given (it is stored in the file `affinitymatrix.csv`). Again, we'll see later how to solve the econometrics problem of estimating $A$.

In [3]:
nbcar = 10
affmat = affdf.iloc[0:nbcar,1:nbcar+1].values

In [4]:
sdX = data_X.std().values
sdY = data_Y.std().values
mX = data_X.mean().values
mY = data_Y.mean().values

Xvals = ((data_X-mX)/sdX).values
Yvals = ((data_Y-mY)/sdY).values

nobs = Xvals.shape[0]
Phi = Xvals @ affmat @ Yvals.T

This problem of computation of the Optimal Assignment Problem, more specifically of $\left(\pi,u,v\right)$, is arguably the most studied problem in Computer Science, and dozens, if not hundreds of algorithms exist, whose running time is polynomial in $\max\left(n,m\right)$, typically a power less than three of the latter.

Famous algorithms include: the Hungarian algorithm (Kuhn-Munkres); Bertsekas' auction algorithm; Goldberg and Kennedy's pseudoflow algorithm. For more on these, see the book by Burkard, Dell'Amico, and Martello, and a
introductory presentation in http://www.assignmentproblems.com/doc/LSAPIntroduction.pdf.

Here, we will show how to solve the problem with the help of a Linear Programming solver used as a black box; our challenge here will be to carefully set up the constraint matrix as a sparse matrix in order to let a large scale Linear Programming solvers such as Gurobi recognize and exploit the sparsity of the problem.

## Computation using linear programming

Let $\Pi$ and $\Phi$ be the matrices with typical elements $\left(\pi_{xy}\right)  $ and $\left(  \Phi_{xy}\right)  $. We let $p$, $q$, $u$,$v$, and $1$ the column vectors with entries $\left(  p_{x}\right)$, $\left(  q_{y}\right)  $, $\left(  u_{x}\right)  $, $\left(  v_{y}\right)$, and $1$, respectively. The optimal assignment problem

\begin{align*}
\max_{\pi\geq0}  &  \sum_{xy}\pi_{xy}\Phi_{xy} \\
s.t.~  &  \sum_{y\in\mathcal{Y}}\pi_{xy}=p_{x}~\left[  u_{x}\right]\\
&  \sum_{x\in\mathcal{X}}\pi_{xy}=q_{y}~\left[  v_{y}\right] 
\end{align*}

Can be rewritten writes using matrix algebra as

\begin{align*}
&  \max_{\Pi\geq0}Tr\left(  \Pi^{\top}\Phi\right) \\
&  \Pi1_{M}=p\\
&  1_{N}^{\top}\Pi=q^{\top}.
\end{align*}

For this we shall need to *vectorize* matrices $\Pi$ and $\Phi$. We need to understand how Python represents matrices in the memory.

### Vectorization

By default, Python uses the *the row-major order*: a matrix is represented as the concatenation of its rows. Indeed:

In [5]:
test = np.array([[1,2,3],[4,5,6]])
test.flatten()

array([1, 2, 3, 4, 5, 6])

### Kronecker product


Loosely speaking, the Kronecker product allows us to understand how a matrix product is vectorized. More precisely, if $A$ is a $M\times p$ matrix and $B$ a $N\times q$ matrix, and if $vec(A)$ vectorizes in the row-major order (i.e. concatenates the rows of $A$) then the Kronecker product $A\otimes B$ of $A$ and $B$ is a $mn\times pq$ matrix such that

\begin{align*}
vec\left(  AXB^{\top}\right)  =\left(  A\otimes B\right)  vec\left(
X\right). \label{VecAndKronecker}
\end{align*}

In python, $A\otimes B$ is implemented by `sparse.kron(A,B)` of the library `sparse` of scipy.

The first constraint $I_{N}\Pi1_{M}=p$,  therefore vectorizes under the row-major order as

\begin{align*}
\left( I_{N} \otimes1_{M}^{\top}\right)  vec\left(  \Pi\right)  =vec\left(
p\right),
\end{align*}

and similarly, the second constraint $1_{N}^{\top}\Pi I_{M}=q^{\top}$, vectorizes as

\begin{align*}
\left( 1_{N}^{\top} \otimes I_{M}\right)  vec\left(  \Pi\right)  =vec\left(
q\right)  .
\end{align*}

Note that the matrix $I_{N} \otimes1_{M}^{\top}$ is of size $N\times NM$, and the matrix $ 1_{N}^{\top} \otimes I_{M}$ is of size $M\times NM$; hence the full matrix involved in the left-hand side of the constraints is of size $\left(  N+M\right)  \times NM$. In spite of its large size, this matrix is *sparse*. In python, the identity matrix $I_{N}$ is coded as `scipy.sparse.identity(N)`.

In [6]:
N,M = Phi.shape

A1 = spr.kron(spr.identity(N),np.array(np.repeat(1,M)))
A2 = spr.kron(np.array(np.repeat(1,N)),spr.identity(M))
A = spr.vstack([A1, A2])

p,q = np.repeat(1/nobs, nobs), np.repeat(1/nobs, nobs)
d = np.concatenate((p,q), axis = None)

Setting $z=vec\left(  \Pi\right)$, the Linear Programming problem then becomes

\begin{align*}
&  \max_{z\geq0}vec\left(  \Phi\right)  ^{\top}z\label{LPvectorized}\\
s.t.~  &  \left( I_{N} \otimes1_{M}^{\top}\right)  vec\left(  \Pi\right)  =vec\left(
p\right)
\nonumber\\
&  \left( 1_{N}^{\top} \otimes I_{M}\right)  vec\left(  \Pi\right)  =vec\left(
q\right) 
\nonumber
\end{align*}

which is ready to be passed on to a linear programming solver such as Gurobi.

A LP solver typically computes programs of the form

\begin{align*}
&  \max_{z\geq0}c^{\top}z\label{standardLP}\\
&  s.t.~Az=d.\nonumber
\end{align*}

In [7]:
m=grb.Model()
x = m.addMVar(shape=N*M, name="x")
m.setObjective(Phi.flatten() @ x, grb.GRB.MAXIMIZE)
m.addConstr(A @ x == d)
m.optimize()

if m.status == grb.GRB.Status.OPTIMAL:
    solution = np.array(m.getAttr('x')).reshape(N,M)
    pi = m.getAttr('pi')

Using license file C:\Users\Alfred\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 2316 rows, 1340964 columns and 2681928 nonzeros
Model fingerprint: 0x5c3c7a75
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-07, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-04, 9e-04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 3.56s
Presolved: 2316 rows, 1340964 columns, 2681928 nonzeros

Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 1.341e+06
 Factor NZ  : 2.683e+06 (roughly 600 MBytes of memory)
 Factor Ops : 4.144e+09 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.14844850e-13  6.69685424e-20  1.16e+02 7.60e+00  1.68e+00     5s
   1  -1.11574956e+02  1.69369750e+01

Which partner does man $i_0$ match with?

In [8]:
print('Man i_0 matches with woman j_'+str (np.argwhere(solution[0,:] != 0)[0][0] )+'.')

Man i_0 matches with woman j_575.


# Part 2: optimal transport with entropic regularization

### Learning objectives

* Entropic regularization

* The log-sum-exp trick

* Gradient descent, coordinate descent

* The Iterated Proportional Fitting Procedure (IPFP)

### References

* Galichon, *Optimal Transport Methods in Economics*,  Ch. 7.3

* Peyré, Cuturi, *Computational Optimal Transport*, Ch. 4.

### Entropic regularization of the optimal transport problem

Consider the problem

\begin{align*}
\max_{\pi\in\mathcal{M}\left(  p,q\right)  }\sum_{ij}\pi_{ij}\Phi_{ij}-\sigma\sum_{ij}\pi_{ij}\ln\pi_{ij}
\end{align*}

where $\sigma>0$. The problem coincides with the optimal assignment problem when $\sigma=0$. When $\sigma\rightarrow+\infty$, the solution to this problem approaches the independent coupling, $\pi_{ij}=p_{i}q_{j}$.

Later on, we will provide microfoundations for this problem, and connect it with a number of important methods in economics (BLP, gravity model, Choo-Siow...). For now, let's just view this as an extension of the optimal transport problem.

Note that in the above, Gurobi is for benchmark purposes with the case $\sigma=0$, but is not suited to compute the nonlinear optimization problem above.

---

We will use the same $\Phi_{ij}$ as in the previous block. When running comparisons we will work on a smaller population, with `nbX` types of men and `nbY` types of women.

In [9]:
nbX = 5
nbY = 3
tol = 1e-9
maxite = 1e+06

Phi_0 = (Xvals @ affmat @ Yvals.T)[:nbX,:nbY]
    
p_0 = np.repeat(1/nbX, nbX)
q_0 = np.repeat(1/nbY, nbY)

nrow,ncol = min(8, nbX),min(8, nbY) # number of rows/columns to display

As a warm-up, let us compute as in the previous part the solution to the problem for $\sigma=0$ that we can compute with Gurobi:

In [10]:
ptm = time.time()

A1_0 = spr.kron(spr.identity(nbX),np.array(np.repeat(1,nbY)))
A2_0 = spr.kron(np.array(np.repeat(1,nbX)),spr.identity(nbY))
A_0 = spr.vstack([A1_0, A2_0])

    
m = grb.Model()
x = m.addMVar(nbX*nbY, name='x')
m.setObjective(Phi_0.flatten() @ x, grb.GRB.MAXIMIZE)
m.addConstr(A_0 @ x == np.hstack([p_0,q_0]))
m.setParam( 'OutputFlag', False ) #quiet output
m.optimize()
diff = time.time() - ptm
print('Time elapsed (Gurobi) = ', diff, 's.')
if m.status == grb.GRB.Status.OPTIMAL:
    val_gurobi = m.objval
    x = m.getAttr('x')
    x = np.array(x).reshape([nbX, nbY])
    pi = m.getAttr('pi')
    u_gurobi = pi[:nbX]
    v_gurobi = pi[nbX:nbX + nbY]
    print("Value of the problem (Gurobi) = ", val_gurobi)
    print(np.subtract(u_gurobi[:nrow], u_gurobi[nrow - 1]))
    print(np.add(v_gurobi[:ncol], u_gurobi[nrow - 1]))
    print('*************************')

Time elapsed (Gurobi) =  0.7829978466033936 s.
Value of the problem (Gurobi) =  0.41095324822187473
[-0.63237262 -0.57124993  1.634949   -1.24417523  0.        ]
[0.62598494 0.61304671 0.48153736]
*************************


### Dual of the regularized problem

Let's compute the dual by the minimax approach. We have

\begin{align*}
\max_{\pi\geq0}\min_{u,v}\sum_{ij}\pi_{ij}\left(  \Phi_{ij}-u_{i}-v_{j}%
-\sigma\ln\pi_{ij}\right)  +\sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}%
\end{align*}

thus

\begin{align*}
\min_{u,v}\sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}+\max_{\pi\geq0}\sum_{ij}%
\pi_{ij}\left(  \Phi_{ij}-u_{i}-v_{j}-\sigma\ln\pi_{ij}\right)
\end{align*}

By FOC in the inner problem, one has $\Phi_{ij}-u_{i}-v_{j}-\sigma\ln \pi_{ij}-\sigma=0,$thus

\begin{align*}
\pi_{ij}=\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}-\sigma}{\sigma}\right)
\end{align*}

and $\pi_{ij}\left(  \Phi_{ij}-u_{i}-v_{j}-\sigma\ln\pi_{ij}\right) =\sigma\pi_{ij}$, thus the dual problem is

\begin{align*}
\min_{u,v}\sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}+\sigma\sum_{ij}\exp\left(
\frac{\Phi_{ij}-u_{i}-v_{j}-\sigma}{\sigma}\right)  .
\end{align*}

After replacing $v_{j}$ by $v_{j}+\sigma$, the dual is

\begin{align*}
\min_{u,v}\sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}+\sigma\sum_{ij}\exp\left(
\frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)  -\sigma. \tag{V1}
\end{align*}

### Another expression of the dual

**Claim:** the problem is equivalent to

<a name='V2'></a>
\begin{align*}
\min_{u,v}\sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}+\sigma\log\sum_{i,j}
\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)  \tag{V2}
\end{align*}

Indeed, let us go back to the minimax expression

\begin{align*}
\min_{u,v}\sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}+\max_{\pi\geq0}\sum_{ij}\pi_{ij}\left(  \Phi_{ij}-u_{i}-v_{j}-\sigma\ln\pi_{ij}\right)
\end{align*}

we see that the solution $\pi$ has automatically $\sum_{ij}\pi_{ij}=1$; thus we can incorporate the constraint into

\begin{align*}
\min_{u,v}\sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}+\max_{\pi\geq0:\sum_{ij}\pi_{ij}=1}\sum_{ij}\pi_{ij}\left(  \Phi_{ij}-u_{i}-v_{j}-\sigma\ln\pi_{ij}\right)
\end{align*}

which yields the [our desired result](#V2).

[This expression](#V2) is interesting because, taking *any* $\hat{\pi}\in
M\left(  p,q\right)$, it reexpresses as

\begin{align*}
\max_{u,v}\sum_{ij}\hat{\pi}_{ij}\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)  -\log\sum_{ij}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)
\end{align*}

therefore if the parameter is $\theta=\left(  u,v\right)$, observations are
$ij$ pairs, and the likelihood of $ij$ is

\begin{align*}
\pi_{ij}^{\theta}=\frac{\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma
}\right)  }{\sum_{ij}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)
}
\end{align*}

Hence, [our expression](#problem) will coincide with the maximum likelihood in this model.

### A third expression of the dual problem

Consider

<a name='V2'></a>
\begin{align*}
\min_{u,v}  &  \sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j} \\
s.t. \quad &  \sum_{i,j}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)
=1
\end{align*}

It is easy to see that the solutions of this problem coincide with [version 2](#V2). Indeed, the Lagrange multiplier is forced to be one. In other words,

\begin{align*}
\min_{u,v}  &  \sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}\\
s.t. \quad &  \sigma\log\sum_{i,j}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma
}\right)  =0
\end{align*}

### Small-temperature limit and the log-sum-exp trick

Recall that when $\sigma\rightarrow0$, one has

\begin{align*}
\sigma\log\left(  e^{a/\sigma}+e^{b/\sigma}\right)  \rightarrow\max\left(
a,b\right)
\end{align*}

Indeed, letting $m=\max\left(  a,b\right)$,

<a name='lse'></a>
\begin{align*}
\sigma\log\left(  e^{a/\sigma}+e^{b/\sigma}\right)  =m+\sigma\log\left(\exp\left(  \frac{a-m}{\sigma}\right)  +\exp\left(  \frac{b-m}{\sigma}\right)\right),
\end{align*}
and the argument of the logarithm lies between $1$ and $2$.

This simple remark is actually a useful numerical recipe called the *log-sum-exp trick*: when $\sigma$ is small, using [the formula above](#lse) to compute $\sigma\log\left(  e^{a/\sigma}+e^{b/\sigma}\right)$ ensures the exponentials won't blow up.


### The log-sum-exp trick for regularized OT

Back to the third expression, with $\sigma\rightarrow0$, one has

\begin{align*}
\min_{u,v}  &  \sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}\tag{V3}\\
s.t.  &  \max_{ij}\left(  \Phi_{ij}-u_{i}-v_{j}\right)  =0\nonumber
\end{align*}

This is exactly equivalent with the classical Monge-Kantorovich expression

\begin{align*}
\min_{u,v}  &  \sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}\tag{V3}\\
s.t.  &  \Phi_{ij}-u_{i}-v_{j}\leq0\nonumber
\end{align*}

Back to the third expression of the dual, with $\sigma\rightarrow0$, one has

\begin{align*}
\min_{u,v}  &  \sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}\tag{V3}\\
s.t.  &  \max_{ij}\left(  \Phi_{ij}-u_{i}-v_{j}\right)  =0\nonumber
\end{align*}

This is exactly equivalent with the classical Monge-Kantorovich expression

\begin{align*}
\min_{u,v}  &  \sum_{i}u_{i}p_{i}+\sum_{j}v_{j}q_{j}\tag{V3}\\
s.t.  &  \Phi_{ij}-u_{i}-v_{j}\leq0\nonumber
\end{align*}

### Computation

We can compute $\min F\left(  x\right)$ by two methods:

Either by gradient descent: $x\left(  t+1\right)  =x_{t}-\epsilon _{t}\nabla F\left(  x_{t}\right)  $. (Steepest descent has $\epsilon _{t}=1/\left\vert \nabla F\left(  x_{t}\right)  \right\vert $.)

Or by coordinate descent: $x_{i}\left(  t+1\right)  =\arg\min_{x_{i}}F\left(  x_{i},x_{-i}\left(  t\right)  \right)$.

Why do these methods converge? Let's provide some justification. We will decrease $x_{t}$ by $\epsilon d_{t}$, were $d_{t}$ is normalized by $\left\vert d_{t}\right\vert _{p}:=\left(  \sum_{i=1}^{n}d_{t}^{i}\right) ^{1/p}=1$. At first order, we have 

\begin{align*}
F\left(  x_{t}-\epsilon d_{t}\right)  =F\left(  x_{t}\right)  -\epsilon d_{t}^{\intercal}\nabla F\left(  x_{t}\right)  +O\left(  \epsilon^{1}\right).
\end{align*}

We need to maximize $d_{t}^{\intercal}\nabla F\left(  x_{t}\right)$ over $\left\vert d_{t}\right\vert _{p}=1$.

* For $p=2$, we get $d_{t}=\nabla F\left(  x_{t}\right)  /\left\vert \nabla F\left(  x_{t}\right)  \right\vert $

* For $p=1$, we get $d_{t}=sign\left(  \partial F\left(  x_{t}\right)/\partial x^{i}\right)  $ if $\left\vert \partial F\left(  x_{t}\right) /\partial x^{i}\right\vert =\max_{j}\left\vert \partial F\left(  x_{t}\right) /\partial x^{j}\right\vert $, $0$ otherwise.


In our context, gradient descent is

\begin{align*}
u_{i}\left(  t+1\right)    & =u_{i}\left(  t\right)  -\epsilon\frac{\partial
F}{\partial u_{i}}\left(  u\left(  t\right)  ,v\left(  t\right)  \right)
,\text{ and }\\
v_{j}\left(  t+1\right)    & =v_{j}\left(  t\right)  -\epsilon\frac{\partial
F}{\partial v_{j}}\left(  u\left(  t\right)  ,v\left(  t\right)  \right)
\end{align*}

while coordinate descent is

\begin{align*}
\frac{\partial F}{\partial u_{i}}\left(  u_{i}\left(  t+1\right)
,u_{-i}\left(  t\right)  ,v\left(  t\right)  \right)  =0,\text{ and }
\frac{\partial F}{\partial v_{j}}\left(  u\left(  t\right)  ,v_{j}\left(
t+1\right)  ,v_{-j}\left(  t\right)  \right)  =0.
\end{align*}

### Gradient descent

Gradient of objective function in version 1 of our problem:

\begin{align*}
\left(  p_{i}-\sum_{j}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)
,q_{j}-\sum_{i}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)
\right)
\end{align*}

Gradient of objective function in version 2

\begin{align*}
\left(  p_{i}-\frac{\sum_{j}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma
}\right)  }{\sum_{ij}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)
},q_{j}-\frac{\sum_{i}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)
}{\sum_{ij}\exp\left(  \frac{\Phi_{ij}-u_{i}-v_{j}}{\sigma}\right)  }\right)
\end{align*}

### Coordinate descent

Coordinate descent on objective function in version 1:

\begin{align*}
p_{i}  & =\sum_{j}\exp\left(  \frac{\Phi_{ij}-u_{i}\left(  t+1\right)
-v_{j}\left(  t\right)  }{\sigma}\right)  ,\\
q_{j}  & =\sum_{i}\exp\left(  \frac{\Phi_{ij}-u_{i}\left(  t\right)
-v_{j}\left(  t+1\right)  }{\sigma}\right)
\end{align*}

that is

\begin{align*}
\left\{
\begin{array}
[c]{c}
u_{i}\left(  t+1\right)  =\sigma\log\left(  \frac{1}{p_{i}}\sum_{j}\exp\left(
\frac{\Phi_{ij}-v_{j}\left(  t\right)  }{\sigma}\right)  \right)  \\
v_{j}\left(  t+1\right)  =\sigma\log\left(  \frac{1}{q_{j}}\sum_{i}\exp\left(
\frac{\Phi_{ij}-u_{i}\left(  t\right)  }{\sigma}\right)  \right)
\end{array}
\right.
\end{align*}

this is called the Iterated Fitting Proportional Procedure (IPFP), or Sinkhorn's algorithm.

Coordinate descent on objective function in version 2 does not yield a closed-form expression.

### IPFP, matrix version

Letting $a_{i}=\exp\left(  -u_{i}/\sigma\right)  $ and $b_{j}=\exp\left(  -v_{j}/\sigma\right)  $ and $K_{ij}=\exp\left(  \Phi_{ij}/\sigma\right)  $, one has $\pi_{ij}=a_{i}b_{j}K_{ij}$, and the procedure reexpresses as

\begin{align*}
\left\{
\begin{array}
[c]{l}%
a_{i}\left(  t+1\right)  =p_{i}/\left(  Kb\left(  t\right)  \right)
_{i}\text{ and }\\
b_{j}\left(  t+1\right)  =q_{j}/\left(  K^{\intercal}a\left(  t\right)
\right)  _{j}.
\end{array}
\right.
\end{align*}

Because this algorithm involves matrix operations only, and is naturally suited for parallel computation, GPUs are a tool of choice for addressing is. See chap. 4 of Peyré and Cuturi.

**Implementation**. Let's implement this algorithm. 


We define for convenience:

In [11]:
def two_d(X):
    return np.reshape(X,(X.size, 1))

Returning to the matrix-IPFP algorithm:

In [12]:
## Matrix IPFP
ptm = time.time()
ite = 0
sigma = 0.1

K = np.exp(Phi_0/sigma)
B = two_d(np.repeat(1, nbY))
error = tol + 1
    
while error > tol and ite < maxite:
    A = two_d(p_0/(K @ B).flatten(order='F'))
    KA = (A.T @ K)
    error = np.max(abs(np.multiply(KA,B.flatten()/q_0)-1))
    B = (q_0 / KA).T
    ite = ite + 1
        
u = - sigma * np.log(A)
v = - sigma * np.log(B)
pi = (K * A) * np.repeat(B, 5, axis = 1).T
val = np.sum(pi * Phi_0) - sigma * np.sum(pi * np.log(pi))
end = time.time() - ptm
if ite >= maxite:
    print('Maximum number of iteations reached in IPFP1.')
else:
    print('IPFP1 converged in ', ite, ' steps and ', end, 's.')
    print('Value of the problem (IPFP1) = ', val)
    print('Sum(pi*Phi) (IPFP1) = ', np.sum(np.multiply(pi,Phi_0)))
    print('*************************')

IPFP1 converged in  89  steps and  0.011997461318969727 s.
Value of the problem (IPFP1) =  0.6045556509904391
Sum(pi*Phi) (IPFP1) =  0.4001284575694893
*************************


To see the benefit of the matrix version, let us recode the same algorithm as above, but in the log-domain, namely iterate over 

In [13]:
# log-domain IPFP
sigma = 0.01
ptm = time.time()
ite = 0
v = np.repeat(0, nbY)
mu = - sigma * np.log(p_0)
nu = - sigma * np.log(q_0)
error = tol + 1
while error > tol and ite < maxite:
    u = mu + sigma * np.log(np.sum(np.exp((Phi_0 - np.repeat(two_d(v), nbX, axis = 1).T)/sigma), axis=1))
    KA = np.sum(np.exp((Phi_0 - two_d(u)) / sigma), axis=0)
    error = np.max(np.abs(KA * np.exp(-v / sigma) / q_0 - 1))
    v = nu + sigma * np.log(KA)
    ite = ite + 1
pi = np.exp((Phi_0 - two_d(u) - np.repeat(two_d(v), nbX, axis = 1).T) / sigma)
val = np.sum(pi * Phi_0) - sigma * np.sum(pi * np.log(pi))
end = time.time() - ptm

if ite >= maxite:
    print('Maximum number of iteations reached in IPFP1.')
else:
    print('IPFP1_logs converged in ', ite, ' steps and ', end, 's.')
    print('Value of the problem (IPFP1_logs) = ', val)
    print('Sum(pi*Phi) (IPFP1_logs) = ', np.sum(pi *Phi_0))
    print('*************************')

IPFP1_logs converged in  156  steps and  0.018995285034179688 s.
Value of the problem (IPFP1_logs) =  0.42959369459249064
Sum(pi*Phi) (IPFP1_logs) =  0.4109531251395
*************************


We see that the log-domain IPFP, while  mathematically equivalent to matrix IPFP, it is noticeably slower. 

### IPFP with the log-sum-exp trick

The matrix IPFPis very fast, partly due to the fact that it involves linear algebra operations. However, it breaks down when $\sigma$ is small; this is best seen taking a log transform and returning to $u^{k}=-\sigma\log a^{k}$ and $v^{k}=-\sigma\log b^{k}$, that is

\begin{align*}
\left\{
\begin{array}
[c]{l}%
u_{i}^{k}=\mu_{i}+\sigma\log\sum_{j}\exp\left(  \frac{\Phi_{ij}-v_{j}^{k-1}%
}{\sigma}\right) \\
v_{j}^{k}=\zeta_{j}+\sigma\log\sum_{i}\exp\left(  \frac{\Phi_{ij}-u_{i}^{k}%
}{\sigma}\right)
\end{array}
\right.
\end{align*}

where $\mu_{i}=-\sigma\log p_{i}$ and $\zeta_{j}=-\sigma\log q_{j}$.

One sees what may go wrong: if $\Phi_{ij}-v_{j}^{k-1}$ is positive in the exponential in the first sum, then the exponential blows up due to the small $\sigma$ at the denominator. However, the log-sum-exp trick can be used in order to avoid this issue.



Consider

\begin{align*}
\left\{
\begin{array}
[c]{l}%
\tilde{v}_{i}^{k}=\max_{j}\left\{  \Phi_{ij}-v_{j}^{k}\right\} \\
\tilde{u}_{j}^{k}=\max_{i}\left\{  \Phi_{ij}-u_{i}^{k}\right\}
\end{array}
\right.
\end{align*}

(the indexing is not a typo: $\tilde{v}$ is indexed by $i$ and $\tilde{u}$ by $j$).

One has

\begin{align*}
\left\{
\begin{array}
[c]{l}%
u_{i}^{k}=\mu_{i}+\tilde{v}_{i}^{k-1}+\sigma\log\sum_{j}\exp\left(  \frac
{\Phi_{ij}-v_{j}^{k-1}-\tilde{v}_{i}^{k}}{\sigma}\right) \\
v_{j}^{k}=\zeta_{j}+\tilde{u}_{j}^{k}+\sigma\log\sum_{i}\exp\left(  \frac
{\Phi_{ij}-u_{i}^{k}-\tilde{u}_{j}^{k}}{\sigma}\right)
\end{array}
\right.
\end{align*}

and now the arguments of the exponentials are always nonpositive, ensuring the exponentials don't blow up.

Both the matrix version and the log-domain version of the IPFP  will break down when $\sigma$ is small, e.g. $\sigma=0.001$ (Try!). However if we modify the second procedure using the log-sum-exp trick, things work again:

In [14]:
# IPFP with log-sum-exp trick

sigma = 0.001
ptm = time.time()
ite = 0
v = np.repeat(0, nbY)
mu = - sigma * np.log(p_0)
nu = - sigma * np.log(q_0)
error = tol + 1
uprec = np.NINF
while error > tol and ite < maxite:
    vstar = np.max(Phi_0.T - two_d(v), axis = 0)
    u = mu + vstar + sigma * np.log(np.sum(np.exp((Phi_0 - np.repeat(two_d(v), nbX, axis = 1).T - 
                                                   two_d(vstar))/sigma), axis=1))
    error = np.max(abs(u - uprec))
    uprec = u
    ustar = np.max(Phi_0 - two_d(u), axis = 0)
    KA = np.sum(np.exp((Phi_0 - two_d(u) - np.repeat(two_d(ustar), nbX, axis = 1).T) / sigma), axis=0)

    v = nu + ustar + sigma * np.log(KA)
    ite = ite + 1
pi = np.exp((Phi_0 - two_d(u) - np.repeat(two_d(v), nbX, axis = 1).T) / sigma)
val = np.sum(pi * Phi_0) - sigma * np.sum(pi * np.log(pi))
end = time.time() - ptm

if ite >= maxite:
    print('Maximum number of iteations reached in IPFP1.')
else:
    print('IPFP1_logs converged in ', ite, ' steps and ', end, 's.')
    print('Value of the problem (IPFP1_logs) = ', val)
    print('Sum(pi*Phi) (IPFP1_logs) = ', np.sum(pi *Phi_0))
    print('*************************')

IPFP1_logs converged in  315  steps and  0.05800271034240723 s.
Value of the problem (IPFP1_logs) =  nan
Sum(pi*Phi) (IPFP1_logs) =  0.4109535261379549
*************************


C:\Users\Alfred\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
C:\Users\Alfred\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in multiply
